In [25]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [26]:
history_df = pd.read_json('data/streaming_history.json')
# print(history_df.head())

In [27]:
history_df.to_csv('data/streaming_history.csv', index=False)
print(history_df.shape)

(13509, 21)


In [28]:
history_df.info()
# We can probably drop episode name, episode show name and episode uri

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13509 entries, 0 to 13508
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ts                                 13509 non-null  object 
 1   username                           13509 non-null  object 
 2   platform                           13509 non-null  object 
 3   ms_played                          13509 non-null  int64  
 4   conn_country                       13509 non-null  object 
 5   ip_addr_decrypted                  13509 non-null  object 
 6   user_agent_decrypted               11607 non-null  object 
 7   master_metadata_track_name         13402 non-null  object 
 8   master_metadata_album_artist_name  13402 non-null  object 
 9   master_metadata_album_album_name   13402 non-null  object 
 10  spotify_track_uri                  13402 non-null  object 
 11  episode_name                       20 non-null     obj

In [29]:
history_df = history_df.drop(columns=['episode_name', 'episode_show_name', 'spotify_episode_uri'])
# history_df.info()
history_df.nunique()

ts                                   13126
username                                 1
platform                                 9
ms_played                             8532
conn_country                             4
ip_addr_decrypted                      674
user_agent_decrypted                    11
master_metadata_track_name            2900
master_metadata_album_artist_name      779
master_metadata_album_album_name      1713
spotify_track_uri                     3147
reason_start                             8
reason_end                              10
shuffle                                  2
skipped                                  2
offline                                  2
offline_timestamp                    12964
incognito_mode                           2
dtype: int64